In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv', na_values='nan')
test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv', na_values='nan')
sub = pd.read_csv('/kaggle/input/playground-series-s5e1/sample_submission.csv')

In [4]:
#time to julian date
#train
train.date = pd.to_datetime(train['date'])
train['date'] = pd.DatetimeIndex(train['date']).to_julian_date()

#test
test.date = pd.to_datetime(test['date'])
test['date'] = pd.DatetimeIndex(test['date']).to_julian_date()

In [5]:
def convert_cat(df, cat):
    for cat in cat:
        df[cat] = df[cat].astype('category')

In [6]:
cat = ['country', 'store', 'product']
convert_cat(train, cat)
convert_cat(test, cat)

In [7]:
train.dropna(inplace=True)

In [8]:
X = train.drop(['id', 'num_sold'], axis=1)
y = np.log1p(train['num_sold'])
test = test.drop('id', axis=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=96)

In [10]:
model = XGBRegressor( verbosity = 1,
                     enable_categorical = True,
                     missing=np.nan
)
model.fit(X_train,y_train, eval_set = [(X_test,y_test)])

[0]	validation_0-rmse:1.06977
[1]	validation_0-rmse:0.76082
[2]	validation_0-rmse:0.54777
[3]	validation_0-rmse:0.40294
[4]	validation_0-rmse:0.30570
[5]	validation_0-rmse:0.24104
[6]	validation_0-rmse:0.19987
[7]	validation_0-rmse:0.17383
[8]	validation_0-rmse:0.15914
[9]	validation_0-rmse:0.14987
[10]	validation_0-rmse:0.14411
[11]	validation_0-rmse:0.14090
[12]	validation_0-rmse:0.13796
[13]	validation_0-rmse:0.13528
[14]	validation_0-rmse:0.13409
[15]	validation_0-rmse:0.13221
[16]	validation_0-rmse:0.13145
[17]	validation_0-rmse:0.13027
[18]	validation_0-rmse:0.12959
[19]	validation_0-rmse:0.12720
[20]	validation_0-rmse:0.12583
[21]	validation_0-rmse:0.12530
[22]	validation_0-rmse:0.12334
[23]	validation_0-rmse:0.12232
[24]	validation_0-rmse:0.12061
[25]	validation_0-rmse:0.11943
[26]	validation_0-rmse:0.11851
[27]	validation_0-rmse:0.11836
[28]	validation_0-rmse:0.11802
[29]	validation_0-rmse:0.11560
[30]	validation_0-rmse:0.11476
[31]	validation_0-rmse:0.11279
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
#predict = model.predict(X_test)
#mape = mean_absolute_percentage_error(np.expm1(predict), np.expm1(y_test))
#mape

In [12]:
yhat = model.predict(test)

In [13]:
sub['num_sold'] = np.expm1(yhat)

In [14]:
sub.to_csv('submission.csv', index=False)